# Deep Reinforcement Learning

## Mahdi AliNejad: 401106266
## Saeed Forati: 401106299
## Amirhossein Souri: 401106182

 Reinforcement Learning (RL) is an approach wherein an agent learns to make sequential decisions by interacting with an environment. The objective is for the agent to maximize the cumulative reward it receives over time.
 The agent goes through this process by repeatedly evaluating the consequences of its actions, trying to select actions that lead to better outcomes.

To do this, we will use Gym, an platform for developing and comparing reinforcement learning algorithms. Gym provides an interface for interacting with different environments, it accepts actions from agents and plays them out in an environment, providing rewards.


## Environment

We will be using `CartPole` environment from gym's library for this assignment.  In this environment, a pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

You can use the code below to run an instance of a random agent in this environment and see the results.

In [51]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

In [63]:

!pip install imageio -qq

In [64]:
import gymnasium as gym
import imageio
import numpy as np
from gymnasium import spaces

We use `gym.make()` to make an instance of a certain environemtn. We can then use `.step()` method which accepts an action as input and performs it. Before that we reset the environment to its initial state by using `.reset()` method.

In [68]:
env_name = 'CartPole-v1'

env = gym.make(env_name, render_mode="rgb_array_list")

env.reset()

frames = []

action = 0
for _ in range(500):
    action = env.action_space.sample()

    obs, reward, is_terminated, is_truncated, _ = env.step(action)
    done = is_terminated or is_truncated
    frames.append(env.render()[0])
    
    if done:
        env.reset()

env.close()
kargs = { 'fps': 25, 'quality': 10, 'macro_block_size': None, 
    'ffmpeg_params': ['-s','600x450'] }
imageio.mimsave('./cartpole.mp4', frames, 'FFMPEG', **kargs)


In [69]:
show_video('./cartpole.mp4')

As you can see, the cart fails to keep the balance of the pole. In the next section we will train an agent to learn how to perform this task.

## Algorithm
We will be using A2C algorithm.

Advantage Actor-Critic (A2C) is a reinforcement learning algorithm.
It consists of an actor (which predicts the best action based on the current state) and a critic (which estimates the state's value function to measure expected future rewards).

We will implement this together step by step.




In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical

import numpy as np
import gymnasium as gym
from collections import deque
from tqdm import tqdm


## Neural Network

Here we design a simple feed forward model to embed the observation from the environment to a hidden layer. We then use two fully connected layers on top of the hidden layer, to predict the next action and estimate the value of current state. This acts as both actor, and critic.

You need to implement the convolutional neural network.


In [56]:
class ActorCritic(nn.Module):
    def __init__(self, hidden_size, num_outputs):
        super(ActorCritic, self).__init__()
        #define your model here
        
        self.actor1 = nn.Linear(4, hidden_size)
        self.actor2 = nn.Linear(hidden_size, num_outputs)
        
        self.critic1 = nn.Linear(4, hidden_size)
        self.critic2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        
        value = self.critic1(x)
        value = F.relu(value)
        value = self.critic2(value)

        actions = self.actor1(x)
        actions = F.relu(actions)
        actions = self.actor2(actions)

        action_probs = F.softmax(actions, dim=-1)
        
        
        return action_probs, value

## A2C

The A2C algorithm aims to jointly train both the actor and the critic to improve the policy. It does this by updating the parameters
of the actor to increase the likelihood of good actions and updating the parameters
of the critic to better estimate the value function.

In each iteration A2C plays the until it ends. During this time it records log probabality of actions, rewards, and predicted values in each step. These values will be used to update the model at the end of this trajectory.

The actor is updated using the objective below:

$$ L_{\text{actor}} = -\log \pi(a|s;\theta) \times A(s, a) $$
Where advantage is calculated as:
$$A(s, a) = Q(s, a) - V(s) $$

Namely the function $Q(s,a)$ is the estimated value of taking action
$a$
 in state
$s$.
$V(s)$ is the predicted value of our critic.

This loss function aims to improve the probability of playing actions that result in higher rewards.

As for the critic the loss function is defined as a simple mean square loss between actual value of an state and the predicted one:

$$ L_{\text{critic}} = \frac{1}{2} ( R - V(s))^2 $$

In [57]:
class A2CAgent:
    def __init__(self, env, num_episodes=1000, max_steps=500, lr=1e-3, gamma=0.99, hidden_size=256):
        self.env = env
        self.num_episodes = int(num_episodes)
        self.max_steps = max_steps
        self.gamma = gamma
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # define your actor critic network, its optimzer and the loss function
        self.policy_net = ActorCritic(hidden_size=hidden_size, num_outputs=2)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.lr)
        self.critic_loss = nn.MSELoss()


    def choose_action(self, state):  
        # choose an action by sampling from the policy distribution
        action_probs, _ =  self.policy_net.forward(state)
        action, _ = self.get_action(action_probs)
        return action

    def compute_returns(self, rewards):
        # calculate the discounted rewards used in the training process
        R = 0
        returns = []
        for r in reversed(rewards):
            R = self.gamma * R + r
            returns.insert(0, R)
            
        returns = torch.tensor(returns, dtype=torch.float32)
        return returns

    def train(self):
        episode_rewards = np.array([])
        with tqdm(range(self.num_episodes)) as pbar:
            for episode in pbar:
                # Implement the training loop
                state = self.env.reset()[0]

                episode_reward = 0
                log_probs = []
                values = []
                rewards = []

                for step in range(self.max_steps):
                    episode_reward, is_terminated, is_truncated, next_state = self.train_each_step(episode_reward, log_probs,
                                                                                       rewards, state, values)
                    state = next_state
                    if is_terminated or is_truncated:
                        break

                episode_rewards = self.update_agent(episode_reward, episode_rewards, log_probs, rewards, values)

                pbar.set_description(f"Episode {episode}, Reward: {np.mean(episode_reward)}")
        self.env.close()
        return episode_rewards

    def update_agent(self, episode_reward, episode_rewards, log_probs, rewards, values):
        episode_rewards = np.append(episode_rewards, episode_reward)
        returns = self.compute_returns(rewards)
        values = torch.cat(values)
        advantage = returns - values
        actor_loss = -(torch.stack(log_probs) * advantage.detach()).mean()
        critic_loss = self.critic_loss(values, returns)
        loss = actor_loss + critic_loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return episode_rewards

    def train_each_step(self, episode_reward, log_probs, rewards, state, values):
        action_probs, value = self.policy_net.forward(state)
        action, policy_dist = self.get_action(action_probs)
        log_prob = policy_dist.log_prob(action)
        next_state, reward, is_terminated, is_truncated, _ = env.step(action.item())
        self.append_to_dict(log_prob, log_probs, reward, rewards, value, values)
        episode_reward += reward
        return episode_reward, is_terminated, is_truncated, next_state

    def append_to_dict(self, log_prob, log_probs, reward, rewards, value, values):
        values.append(value)
        log_probs.append(log_prob)
        rewards.append(reward)

    def get_action(self, action_probs):
        policy_dist = Categorical(action_probs)
        action = policy_dist.sample()
        return action, policy_dist

Define the model and set hyperparameters.

In [58]:
env = gym.make(env_name, render_mode="rgb_array_list")
num_episodes = 500
max_steps = 500
lr = 1e-3
gamma = 0.99
hidden_size = 256

a2c_model = A2CAgent(env, num_episodes, max_steps, lr, gamma, hidden_size)

Train the model.

In [59]:
rewards = a2c_model.train()

Episode 499, Reward: 500.0: 100%|██████████| 500/500 [04:27<00:00,  1.87it/s]


## Evaluation

Use the `choose_action` method of the trained agent to evaluate its performance.

In [71]:
env = gym.make(env_name, render_mode='rgb_array_list') 

model = a2c_model

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# complete the loop and gather the episode reward of each episode as well the frames of the first episode
frames = []
episode_rewards = []

for i in range(num_episodes):
    state = env.reset()[0]
    episode_reward = 0
    
    # TODO 
    # Implement the evaluation loop
    is_terminated = False
    is_truncated = False
    while not is_truncated and not is_terminated:
        action = model.choose_action(state)
        next_state, reward, is_terminated, is_truncated, _ = env.step(action.item())
        episode_reward += reward
        state = next_state
        
        frames.append(env.render()[0])

        

    episode_rewards.append(episode_reward)
    print(f"Episode Reward: {episode_reward}")
env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")
kargs = { 'fps': 25, 'quality': 10, 'macro_block_size': None, 
    'ffmpeg_params': ['-s','600x450'] }
imageio.mimsave('./test.mp4', frames, 'FFMPEG', **kargs)

Episode Reward: 500.0
Episode Reward: 425.0
Episode Reward: 399.0
Episode Reward: 411.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 464.0
Episode Reward: 335.0
Episode Reward: 198.0
Episode Reward: 500.0
Average Reward over 10 episodes: 423.2


In [72]:
show_video('./test.mp4')